In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-30"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
27557,2024-11-30,Sérvia Primeira Liga,15:00,Tamis,Cacak 94,1.85,1.81,156.5,1.84,1.82,-1.5,1.98,1.70,OS8sYhrJ,0.540541,0.552486,0.543478,0.549451,0.093027,163.494,65.875356,0.402922,0.6,1.341641,2.236068,90.39,2.320,1.071331,0.461781,-42.0,108.250,11.473750,0.105993,1.2,1.643168,1.369306,96.72,1.366,0.164408,0.120357,21.0,69,78,1.31,1.24,177.992,207.390,0.015456,0.007728,0.107603,0.96,0.192,4.427083,0.513693,0.5,-0.013693,-0.16,-0.032,-25.312500,0.530542,0.5,-0.030542
27558,2024-11-30,Sérvia 1. Zls Feminino,12:30,ZKK Kraljevo F,Nis F,1.58,2.29,140.5,1.83,1.83,-3.5,1.83,1.83,KSk7rWNE,0.632911,0.436681,0.546448,0.546448,0.069593,283.804,256.977684,0.905476,1.8,1.643168,0.912871,139.40,4.366,3.878863,0.888425,-11.0,155.134,83.913061,0.540907,1.8,1.643168,0.912871,113.40,1.972,1.073601,0.544423,42.0,85,84,1.64,1.35,129.502,321.022,0.259455,0.000000,0.000000,-1.85,-0.370,-1.567568,0.000000,0.0,0.000000,-3.65,-0.730,-1.767123,0.000000,0.0,0.000000
27559,2024-11-30,Sérvia 1. Zls Feminino,11:00,Mega MIS F,Duga F,1.21,3.74,135.5,1.85,1.81,-9.5,1.82,1.84,QRHvQN7m,0.826446,0.267380,0.540541,0.552486,0.093826,128.670,83.010005,0.645139,1.8,1.643168,0.912871,59.28,1.934,1.463328,0.756633,31.0,156.736,104.598636,0.667356,0.6,1.341641,2.236068,125.66,3.066,3.285366,1.071548,-104.0,57,61,1.04,2.06,102.890,156.304,0.722820,0.015456,0.007728,0.51,0.102,2.058824,0.835017,0.9,0.064983,-3.71,-0.742,-3.692722,0.000000,0.0,0.000000
27560,2024-11-30,Sérvia Primeira Liga,15:30,Sloga,Vrsac,2.43,1.47,165.5,1.84,1.82,3.5,2.03,1.67,SMj9nhcC,0.411523,0.680272,0.543478,0.549451,0.091795,265.428,177.514503,0.668786,0.0,0.000000,NaN,549.50,3.600,2.537075,0.704743,-63.0,126.104,12.992047,0.103026,2.4,1.341641,0.559017,103.84,1.468,0.231344,0.157591,33.0,70,88,7.85,1.18,159.802,140.428,0.348114,0.007728,0.137599,-1.83,-0.366,-3.907104,0.575883,0.6,0.024117,-0.27,-0.054,-8.703704,0.572107,0.5,-0.072107
27561,2024-11-30,Suécia Liga De Basquete,11:00,AIK Basket F,Visby F,1.08,5.91,153.5,1.80,1.86,-15.5,1.83,1.83,MTsH3iOO,0.925926,0.169205,0.555556,0.537634,0.095131,1072.486,1153.271709,1.075326,0.6,1.341641,2.236068,299.20,15.584,17.470570,1.121058,-64.0,372.476,276.223392,0.741587,1.2,1.643168,1.369306,286.20,5.924,4.591942,0.775142,-63.0,80,60,3.74,4.77,342.906,885.640,0.977203,0.023184,0.000000,-3.59,-0.718,-0.111421,0.000000,0.0,0.000000,-5.00,-1.000,-4.910000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27872,2024-11-30,França Lfb Feminina,16:00,Chartres F,La Roche F,1.25,3.43,144.5,1.80,1.86,-9.5,1.93,1.74,zoeGeUom,0.800000,0.291545,0.555556,0.537634,0.091545,258.756,123.651483,0.477869,0.6,1.341641,2.236068,380.48,3.712,1.992315,0.536723,-31.0,201.918,52.615057,0.260576,0.0,0.000000,NaN,199.51,3.318,1.200362,0.361773,-81.0,58,71,6.56,2.81,0.000,177.130,0.658758,0.023184,0.073215,0.00,0.000,inf,0.000000,0.0,0.000000,-3.45,-0.690,-3.521739,0.330048,0.2,-0.130048
27873,2024-11-30,França Lfb Feminina,16:00,Bourges F,Lyon F,1.09,5.61,148.5,1.88,1.78,-14.5,2.00,1.69,t4yL2CID,0.917431,0.178253,0.531915,0.561798,0.095684,93.372,8.3460

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
27579,08:00,Turquia Tb2L,Cozum Ankara,Akhisar Belediye,2.75,Back Home
27580,07:30,Turquia Tb2L,Istanbul BB,Konyaspor,1.83,Back Home
27587,09:30,Turquia Superliga,Buyukcekmece,Galatasaray,2.41,Back Home
27606,14:00,Polônia 1. Liga,Poli. Opolska,Polonia Warszawa,1.56,Back Home
27609,13:30,Polônia Liga De Basquete,Legia,Anwil Wloclawek,2.38,Back Home
27637,21:30,Eua Ncaa,Pacific,CS Fullerton,1.69,Back Home
27654,18:30,Eua Ncaa,Pennsylvania,Maine,2.00,Back Home
27678,20:30,Chile Copa Do Chile,Espanol Osorno,Ancud,1.64,Back Home
27692,23:00,Eua Nba,Phoenix Suns,Golden State Warriors,1.63,Back Home
27697,18:00,Eua Ncaa,La Salle,Temple,1.87,Back Home
